<img src="Image 4.png" width="1050" align="center">

Data Pre-Porcessing is a vital step in any Machine Learning or Deep Learning technique. This mining technique which is used to transform the raw data in a useful and efficient format.
In this case we are dealing with Image Data where a model to understand and differentiate an image, certain Pre-Processing techniques are implemented. Such approach helps in efficient Model Building.

The Model will be trained on such Pre-Processed data which can enhance the understanding of type and features of an image.
In this process, the data are made to Pre-Process under various factors and conditions which helps the model to classify the given image being "Accept" or "Reject".

## 1. Importing Libraries

In [1]:
import imgaug as ia                                                     # imgaug uses matplotlib backend for displaying images
ia.seed(1)
%matplotlib inline
import imageio                                                          # Image Input/Output Reader
import pandas as pd                                                     # Importing Pandas
import numpy as np                                                      # Importing Numpy
import os                                                               # Importing Operating System
import glob                                                             # Used to return all file paths that match a specific pattern
from PIL import Image, ImageEnhance                                     # Using Pillow, import Image and Image Enhancer (To enhance the quality of an Image)
from skimage import exposure                                            # Implemented for Histogram Equalization
import matplotlib.pyplot as plt                                         # Importing Matplotlib for Visualization 
import cv2                                                              # Importing Computer Vision Tool
 
img_width, img_height = 256, 256

plt.rcParams['figure.figsize'] = [15,8]                                 # Setting a standard figure size of (15,8)

## 2. Accessing Image

Below the paths of both Cropped images of Accept and Reject are accessed through os.path.join.

The Cropped images are accessed for the analysis which will be further used for Image Pre-Processing

In [3]:
# Creating a New Directory for Accept Images

CROPPED_PATH = os.path.join('reinforced_images4','Accept_Images')
# C:\Users\nabhishe\MITC Project

if not os.path.exists(CROPPED_PATH):
    !mkdir {CROPPED_PATH}

In [ ]:
# Creating a New Directory for Reject Images

CROPPED_PATH = os.path.join('reinforced_images4','Reject_Images')
# C:\Users\nabhishe\MITC Project

if not os.path.exists(CROPPED_PATH):
    !mkdir {CROPPED_PATH}

#### Accept & Reject Image color distribution

Below shows a basic idea of how images are classified based on the coating.

- The images are read from the directory and converted from BGR to RGB.
- An estimation is established by setting a threshold. In this case a threshold of 140 is set (taking a grayscale range from 0 to 255).
- A condition given if mean of an image is greater than 140 [ mean(img) > 140 ]
- If this condition satisfies, the pattern/Coating of an image is considered "Dark" else "Bright"

The above analysis helps in understanding the pixel distribution based on different ways a plate is coated.


In [ ]:
# Reject Images
images = glob.glob("C:\\Users\\nabhishe\\MITC_Project\\cropped_images1\\Reject_Cropped\\*.jpg")
for image in images:
    f = imageio.imread(image, as_gray=True)
    img1 = cv2.imread(image)
    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    plt.show()
    def img_estim(img, thrshld):
        is_light = np.mean(img) > thrshld
        return 'light' if is_light else 'dark'

    print(img_estim(f, 140))

<img src="Image 5.png" width="750" align="center">

A similar approach is taken for Accept Images for understanding the color coating distribution.

In [ ]:
# Accept Images
images = glob.glob("C:\\Users\\nabhishe\\MITC_Project\\cropped_images1\\Accept_Cropped\\*.jpg")
for image in images:
    f = imageio.imread(image, as_gray=True)
    img1 = cv2.imread(image)
    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    plt.show()
    def img_estim(img, thrshld):
        is_light = np.mean(img) > thrshld
        return 'light' if is_light else 'dark'

    print(img_estim(f, 140))

<img src="Image 6.png" width="750" align="center">

## 3. Image Pre-Processing

Image Pre-Processing is a vital step for any Model Building. In order for a model to understand the features and using that to classify, we go with Pre-Processing.
In this stage the images are divided into 4 categories. 

1) Dark Reject Images

2) Bright Reject Images

3) Dark Accept Images

4) Bright Accept Images

The above step is carried out due to variations in Coating of plate. In order for a model to undertand and classify whether a given image is "Accept" or "Reject" irrespective of any type of coating, the above 4 methods are carried.

This step handles biasing of image irrespective of any type of Coating and can enhance the performance of Neural Network Model.

### Reject Images

#### Dark Reject Images

The below implementation helps in understanding the methods taken to handle Dark Reject Images coatings. 

In [ ]:
# considering a Dark Reject image

path = 'C:\\Users\\nabhishe\\MITC_Project\\cropped_images1\\Reject_Cropped\\'
for f in os.listdir(path):
    if f.endswith(".jpg"):
        image = plt.imread('C:\\Users\\nabhishe\\MITC_Project\\cropped_images1\\Reject_Cropped\\'+str(f))
        fn, fext = os.path.splitext(f)
        #enhancer = ImageEnhance.Brightness(image)
        #new_image = enhancer.enhance(1.5)
        threshold = 140
        if np.mean(image) < threshold:
            adapthist_image = exposure.equalize_hist(image)
            im = Image.fromarray((adapthist_image*255).astype(np.uint8)) 
            images = im.convert(mode = 'L')  
            image_np = np.array(images)
            sharp = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])      
            custom_img = cv2.filter2D(image_np,-1,sharp)
            contrast = cv2.convertScaleAbs(custom_img, alpha=1, beta=50)
            blurred = cv2.GaussianBlur(contrast, (3, 3), 0)
            thresh = cv2.threshold(blurred, 120, 255, cv2.THRESH_BINARY)[1]
            thresh = cv2.erode(thresh, None, iterations=7)
            thresh = cv2.dilate(thresh, None, iterations=3)
            cv2.imwrite("C:\\Users\\nabhishe\\MITC_Project\\reinforced_images4\\Reject_Images\\{}.{}".format(fn, fext), thresh)


* <b>Implementation</b> : 

The paths are accessed for the Cropped Reject Images using os.listdir

The images are read from the directory and a condition is set at 140. This condition is set after various tries of ranges based on Grayscale (0 to 255).

- Mean of an image is considered and checked if it is less than <b>threshold (140)</b>
- <b>Adaptive Equalized</b> Threshold is used to check the distribution of pixel across the graph.
- In this case exposure.equalize_hist is implemented to uniform distribution of pixel across the range
- The input image is then converted into an array with a type of <b>8-bit</b> using (.astype).
- Further the image is converted into <b>Grayscale</b> using <b>.convert(mode='L')</b>
- An Array of resultant Grayscale image is Filtered using <b>Filter2D</b> and sharpening with a Kernel by <b>(3x3)</b> matrix.
- <b>ConvertScaleAbs</b> is introduced using cv2 (Computer Vision) and tuning by <b>Alpha and Beta</b>.
- This helps in Scaling, computing absolute values and converts the result to 8-bit.
- The transformed image is passed into <b>Gaussian Blur</b> where the image is blurred to a contrast set by a user defined Kernel, in this case (3,3)
- The Blurred Image is hence Thresholded with <b>THRESH_BINARY</b> and the levels are set with a threshold.
- In the above task each pixel values are compared with the threshold and converted into a Binary Image (Black & White).
- The final Thresholded image is then iterated over <b>Erosion</b> and <b>Dilation</b> Process.
- <b>Erosion</b> and <b>Dilation</b> helps in inceasing or blurring the nature of Pixels.
- Erosion increases the depth in those pixel regions where it is necessary for our analysis.
- Dilation suppresses those pixel regions into white.

<b>Erosion and Dilation helps in fetching those pixels suitable for our Analysis by extracting those Bit mark regions in Black and retaining others in White</b>

#### Bright Reject Images

The similar case of implementation can be used for Bright Reject Images where a Mean of an image is greater than 140 <b>[ mean(img) > 140]</b>

In [ ]:
###########################################################################################

# considering a bright Reject image                                      

path = 'C:\\Users\\nabhishe\\MITC_Project\\cropped_images\\Reject_Cropped\\'
for f in os.listdir(path):
    if f.endswith(".jpg"):
        image = Image.open('C:\\Users\\nabhishe\\MITC_Project\\cropped_images\\Reject_Cropped\\'+str(f))
        #enhancer = ImageEnhance.Brightness(image)
        #im_output = enhancer.enhance(1.5)
        fn, fext = os.path.splitext(f)
        threshold = 140
        if np.mean(image) > threshold:
            images = image.convert(mode = 'L')
            image_np = np.array(images)
            sharp = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])       # sharpening the image
            custom_img = cv2.filter2D(image_np,-1,sharp)
            contrast = cv2.convertScaleAbs(custom_img, alpha=1, beta=60)
            blurred = cv2.GaussianBlur(contrast, (7, 7), 0)
            thresh = cv2.threshold(blurred, 180, 255, cv2.THRESH_BINARY)[1]
            thresh = cv2.erode(thresh, None, iterations=7)
            thresh = cv2.dilate(thresh, None, iterations=3)
            #thresh.save("C:\\Users\\nabhishe\\MITC_Project\\reinforced_images\\Reject_images\\{}_bw.{}".format(fn, fext))
            cv2.imwrite("C:\\Users\\nabhishe\\MITC_Project\\reinforced_images4\\Reject_Images\\{}.{}".format(fn, fext), thresh)

* Implementation :
    
Certain changes are experimented and observed compared to Bright Reject Images. The following are listed below:

- <b>Adaptive Equalized Histogram</b> is not implemented for this analysis.
- The mean of the Image is taken and checked whether it is greater than <b>threshold (140)</b>.
- If the conditions passes, the image is converted into a Grayscale orientation and made it into an Array.
- After adjusting the contrast using <b>ConvertScaleAbs</b>, a Gaussian Blur with kernel <b>(7,7)</b> is used for Image Blurring.
- Since we are dealing with Bright Images, the pixel values of the range <b>180</b> and <b>255</b> are compared with the <b>threshold (140)</b>.
- The resultant image is tuned with <b>Erosion</b> and <b>Dilation</b> and stored in a Directory.

### Accept Images

The Pre-Processing techniques used in Accept Images are dealt with same implementations used for Reject Images.

In this case Accept Images are divided into Dark Accept and Bright Accept based on the Coating. The implementations are carried out in the similar manner as it was dealt with Reject Images.

The Pre-Processing steps taken for both type of Coating are as shown below:

#### Dark Accept Images

In [ ]:
###########################################################################################

# considering a Dark Accept image                                      

path = 'C:\\Users\\nabhishe\\MITC_Project\\cropped_images\\Accept_Cropped\\'
for f in os.listdir(path):
    if f.endswith(".jpg"):
        image = Image.open('C:\\Users\\nabhishe\\MITC_Project\\cropped_images\\Accept_Cropped\\'+str(f))
        fn, fext = os.path.splitext(f)
        threshold = 140
        if np.mean(image) < threshold: 
            images = image.convert(mode = 'L')
            image_np = np.array(images)
            sharp = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])       # sharpening the image
            custom_img = cv2.filter2D(image_np,-1,sharp)
            contrast = cv2.convertScaleAbs(custom_img, alpha=1, beta=110)
            blurred = cv2.GaussianBlur(contrast, (5, 5), 0)
            thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)[1]
            thresh = cv2.erode(thresh, None, iterations=5)
            thresh = cv2.dilate(thresh, None, iterations=5)
            #thresh.save("C:\\Users\\nabhishe\\MITC_Project\\reinforced_images\\Reject_images\\{}_bw.{}".format(fn, fext))
            cv2.imwrite("C:\\Users\\nabhishe\\MITC_Project\\reinforced_images4\\Accept_Images\\{}.{}".format(fn, fext), thresh)

#### Bright Accept Images

In [ ]:
###########################################################################################

# considering a bright Accept image                                       (194 images)

path = 'C:\\Users\\nabhishe\\MITC_Project\\cropped_images\\Accept_Cropped\\'
for f in os.listdir(path):
    if f.endswith(".jpg"):
        image = Image.open('C:\\Users\\nabhishe\\MITC_Project\\cropped_images\\Accept_Cropped\\'+str(f))
        fn, fext = os.path.splitext(f)
        threshold = 140
        if np.mean(image) > threshold:
            images = image.convert(mode = 'L')
            image_np = np.array(images)
            sharp = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])       # sharpening the image
            custom_img = cv2.filter2D(image_np,-1,sharp)
            contrast = cv2.convertScaleAbs(custom_img, alpha=1, beta=100)
            blurred = cv2.GaussianBlur(contrast, (7, 7), 0)
            thresh = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)[1]
            thresh = cv2.erode(thresh, None, iterations=3)
            thresh = cv2.dilate(thresh, None, iterations=5)
            #thresh.save("C:\\Users\\nabhishe\\MITC_Project\\reinforced_images\\Reject_images\\{}_bw.{}".format(fn, fext))
            cv2.imwrite("C:\\Users\\nabhishe\\MITC_Project\\reinforced_images4\\Accept_Images\\{}.{}".format(fn, fext), thresh)